In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, recall_score
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.pipeline import Pipeline

In [3]:
raw_data = pd.read_csv(r'Data.csv')
print(raw_data.shape)
raw_data.head()

C:\Users\Integral\AppData\Local\Temp\ipykernel_19300\2621379761.py:1: DtypeWarning: Columns (35,36,37,38,41,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(r'Data.csv')


(802, 24691)


,sample_id,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,taxid_ch1,characteristics_ch1,treatment_protocol_ch1,growth_protocol_ch1,molecule_ch1,extract_protocol_ch1,label_ch1,label_protocol_ch1,hyb_protocol,scan_protocol,description,data_processing,platform_id,contact_name,contact_email,contact_institute,contact_address,contact_city,contact_zip/postal_code,contact_country,supplementary_file,series_id,data_row_count,gender,age,pneumonia_diagnoses,thrombocytopenia,endotype_cohort,endotype_class,mortality_event_28days,...,200056_PM_s_at,200057_PM_s_at,200058_PM_s_at,200059_PM_s_at,200060_PM_s_at,200061_PM_s_at,200062_PM_s_at,200063_PM_s_at,200064_PM_at,200066_PM_at,200067_PM_x_at,200068_PM_s_at,200069_PM_at,200070_PM_at,200071_PM_at,200072_PM_s_at,200073_PM_s_at,200074_PM_s_at,200075_PM_s_at,200078_PM_s_at,200079_PM_s_at,200081_PM_s_at,200082_PM_s_at,200083_PM_at,200084_PM_at,200085_PM_s_at,200086_PM_s_at,200087_PM_s_at,200088_PM_x_at,200089_PM_s_at,200090_PM_at,200091_PM_s_at,200092_PM_s_at,200093_PM_s_at,200094_PM_s_at,200095_PM_x_at,200096_PM_s_at,200097_PM_s_at,200098_PM_s_at,200099_PM_s_at
0,GSM1602801,3,1602801,6,5,12,RNA,1,whole blood,Homo sapiens,9606,64,4,10,total RNA,Total RNA was isolated in accordance with the ...,biotin,100,"Standard Affymetrix GeneAtlas® Hybridization, ...",Standard GeneTitan® multi channel instrument.,blood gene expression data from healthy subjec...,2.15,13667,"Brendon,,Scicluna",brendon.scicluna@um.edu.mt,University of Malta,Msida campus,Msida,2020,Malta,1602,65682,24646,male,64,no-cap,NaN,NaN,NaN,NaN,...,7.031309,8.809735,6.864026,10.992910,7.525547,9.428671,11.476569,8.127680,5.830617,8.068982,10.192938,8.522552,3.517506,6.725688,6.749709,7.451290,8.601268,9.043899,9.334175,9.477353,6.716201,9.342289,8.935579,5.660123,6.718337,9.545007,8.676797,8.505816,10.805107,9.739911,7.573465,9.842844,11.108367,6.976183,9.982104,10.836103,10.318494,8.732240,6.642293,10.754443
1,GSM1602802,3,1602802,6,5,12,RNA,1,whole blood,Homo sapiens,9606,46,4,10,total RNA,Total RNA was isolated in accordance with the ...,biotin,100,"Standard Affymetrix GeneAtlas® Hybridization, ...",Standard GeneTitan® multi channel instrument.,blood gene expression data from healthy subjec...,2.15,13667,"Brendon,,Scicluna",brendon.scicluna@um.edu.mt,University of Malta,Msida campus,Msida,2020,Malta,1602,65682,24646,female,46,cap,NaN,discovery,Mars2,0.0,...,6.801264,8.762293,6.802847,10.998526,7.628672,9.751905,11.577642,8.957975,6.924858,7.119522,10.712720,8.179706,4.515357,6.359513,7.312598,7.851183,8.887365,9.366163,8.198916,8.852122,7.307672,9.785009,9.505513,6.246581,7.625135,9.042519,8.334976,8.479688,10.992828,9.882581,8.073634,9.813547,11.317698,7.305468,9.986262,10.941711,10.273002,8.531718,7.032988,10.827502
2,GSM1602803,3,1602803,6,5,12,RNA,1,whole blood,Homo sapiens,9606,69,4,10,total RNA,Total RNA was isolated in accordance with the ...,biotin,100,"Standard Affymetrix GeneAtlas® Hybridization, ...",Standard GeneTitan® multi channel instrument.,blood gene expression data from healthy subjec...,2.15,13667,"Brendon,,Scicluna",brendon.scicluna@um.edu.mt,University of Malta,Msida campus,Msida,2020,Malta,1602,65682,24646,male,69,cap,NaN,discovery,Mars2,0.0,...,7.402444,8.684559,6.589870,10.883525,7.048719,10.584323,11.607917,8.756031,6.833993,7.021501,10.976195,8.477087,4.610507,6.455950,7.172838,7.852709,8.614289,9.149657,9.082039,8.994994,6.903686,9.710181,9.700885,5.690542,7.848426,9.289770,8.134887,8.915334,10.856849,9.804579,8.151554,10.039860,11.272069,7.663618,10.266537,10.999459,10.373244,8.038539,6.623814,11.163120
3,GSM1602804,3,1602804,6,5,12,RNA,1,whole blood,Homo sapiens,9606,60,4,10,total RNA,Total RNA was isolated in accordance with the ...,biotin,100,"Standard Affymetrix GeneAtlas® Hybridization, ...",Standard GeneTitan® multi channel instrument.,blood gene expression data from healthy subjec...,2.15,13667,"Brendon,,Scicluna",brendon.scicluna@um.edu.mt,U

In [8]:
expr = pd.read_csv(r'GSE65682_geneCollapsed_BESTprobe_byVAR.csv')
print(expr.shape)
expr.head()

(802, 10871)


,sample_id,200015_PM_s_at,A4GALT,AAA1,AACS,AAGAB,AAK1,AAMP,AARS,AARS2,AASDH,AASDHPPT,AATF,AATK,ABAT,ABCA1,ABCA13,ABCA5,ABCA7,ABCB1,ABCB10,ABCB6,ABCB7,ABCB9,ABCC1,ABCC10,ABCC11,ABCC13,ABCC3,ABCC4,ABCC5,ABCC8,ABCD1,ABCD3,ABCD4,ABCE1,ABCF1,ABCF2,ABCF3,ABCG1,...,ZNF791,ZNF8,ZNF800,ZNF805,ZNF827,ZNF828,ZNF83,ZNF830,ZNF836,ZNF839,ZNF84,ZNF862,ZNF92,ZNF93,ZNFX1,ZNHIT1,ZNHIT3,ZNHIT6,ZNRD1,ZNRF1,ZNRF2,ZP3,ZRANB1,ZRANB2,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN29,ZSCAN5B,ZSWIM6,ZSWIM7,ZUFSP,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,GSM1602801,8.374532,2.476531,3.267279,2.796396,5.062194,6.325879,4.566798,2.702982,2.392874,4.017601,3.849771,5.985365,7.060843,4.750715,6.336179,2.139088,4.533671,4.209780,5.016657,4.211801,2.132111,4.745479,3.341713,5.219530,2.369941,2.132703,2.999418,2.847339,2.529657,5.419211,2.441791,2.751967,4.023322,2.647480,4.171469,4.665715,3.056120,5.007057,4.780844,...,3.941746,3.983654,3.930395,4.315281,2.563315,2.927986,3.397293,3.512724,2.821480,3.852565,2.980733,3.681583,2.159391,3.083249,6.833486,7.100422,3.635303,3.457153,6.568575,3.667565,2.676726,3.020232,4.612177,5.560631,3.846036,3.484608,2.452562,3.048719,2.564449,7.542804,3.867033,2.390572,3.077011,3.192092,2.403091,4.400065,3.965543,7.179740,5.875164,3.085236
1,GSM1602802,8.196153,2.261576,3.104086,3.275543,5.371443,6.391021,4.131982,3.130558,2.481427,4.121449,4.664867,5.335368,6.416242,5.138065,5.810832,4.497633,4.274514,5.481458,4.100326,5.072343,1.986362,4.836463,3.056321,5.418604,2.917964,2.059617,2.383312,2.986157,2.559920,6.069723,2.693091,2.839251,5.375081,2.872539,5.115386,5.089331,3.458103,4.180753,3.874925,...,4.127473,3.539460,4.646045,4.363789,2.415078,3.414326,4.759554,3.820201,2.626380,3.530326,3.022236,4.803728,2.607287,2.637789,6.694000,6.964510,3.950472,4.109149,6.604810,4.093587,3.488449,3.140045,4.493763,6.838405,2.968794,3.409196,2.376050,3.544601,2.396704,7.666054,4.786075,3.155967,3.326524,3.874734,3.921279,3.292959,3.826366,6.528229,5.496339,3.752368
2,GSM1602803,8.049158,2.277492,3.565000,2.753998,5.133350,7.212130,4.197382,2.770229,2.559002,5.238487,5.228219,5.633269,6.157959,5.095889,7.926530,2.182368,4.371890,2.847145,4.619705,5.614970,2.192569,4.214674,3.215800,5.471711,3.076484,1.991024,2.964382,3.151831,2.537996,4.380751,2.495059,2.746748,5.817817,3.262436,5.045266,5.126639,3.201205,4.638202,3.681833,...,4.555637,3.513376,4.574943,4.635307,2.883287,3.188046,4.861153,3.919249,3.025347,3.746914,3.177470,4.704257,2.392589,2.632080,5.661406,7.334993,4.239784,3.185535,7.215705,3.511894,3.291209,3.327510,4.638602,7.119833,2.954438,3.349922,2.835412,2.872529,2.262432,7.290612,4.008079,3.854365,3.431031,3.738531,2.793763,5.173026,4.850186,6.490220,5.241529,3.367788
3,GSM1602804,8.163978,2.203401,3.273304,2.932490,5.445349,7.569249,4.307114,3.692691,3.403940,4.634611,4.208600,6.117894,6.906295,5.494390,5.613944,2.227155,5.193588,5.815250,4.374614,4.300286,2.260918,4.732731,3.295441,5.857176,3.035131,2.061137,2.670072,2.142485,2.681367,5.996883,2.633513,2.937837,4.734822,3.671816,5.228055,5.791337,3.436480,5.063155,4.122469,...,4.169017,4.120427,3.765375,4.113635,3.040914,3.781348,3.874266,3.831988,3.071966,3.797121,3.832514,4.529134,2.299059,2.822191,6.824311,7.065410,4.093865,4.130780,6.868903,3.258425,2.735521,3.124024,4.343564,6.525024,3.374336,4.130425,2.728954,3.219281,2.619333,7.154661,3.783215,2.546389,3.655676,3.334800,2.550041,5.368169,3.939616,6.730774,6.082328,3.226368
4,GSM1602805,7.014305,2.161031,3.847223,2.579574,4.917349,5.910861,3.829845,3.087585,2.796635,3.429107,4.686572,5.110388,7.438322,5.299349,7.086625,2.770758,3.890091,5.908132,4.472770,4.777860,2.319431,3.905190,3.408760,4.657442,2.706301,2.254924,2.546734,2.342180,2.316389,4.927731,2.031095,2.775537,4.890376,2.748090,4.489951,4.192392,2.806805,4.313181,5.517469,...,4.211501,3.184728,4.089738,3.640608,2.629668,2.905217,4.097917,3.354093,2.666624,3.501176,2.746188,4.627008,2.744753,3.145275,6.047121,6.649186,3.946829,3.756474,6.110988,3.185836,3.711202,3.299832,5.00

In [104]:
data = pd.concat([raw_data.iloc[:, :45], expr.iloc[:, 2:]], axis=1)
data = data[(~data['endotype_cohort'].isna()) | (data['icu_acquired_infection'] == 'healthy')]
data['endotype_cohort'] = data['endotype_cohort'].notna().astype(int)
data = data.rename(columns={'endotype_cohort': 'Mortality'})
data = data.dropna(axis=1)
data = data.drop(['sample_id', 'geo_accession'], axis=1)
data = data.iloc[:, ~data.columns.duplicated()]
print(data.shape)
data.head()

(521, 10883)


,title,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,taxid_ch1,characteristics_ch1,treatment_protocol_ch1,growth_protocol_ch1,molecule_ch1,extract_protocol_ch1,label_ch1,label_protocol_ch1,hyb_protocol,scan_protocol,description,data_processing,platform_id,contact_name,contact_email,contact_institute,contact_address,contact_city,contact_zip/postal_code,contact_country,supplementary_file,series_id,data_row_count,gender,age,Mortality,A4GALT,AAA1,AACS,AAGAB,AAK1,AAMP,...,ZNF791,ZNF8,ZNF800,ZNF805,ZNF827,ZNF828,ZNF83,ZNF830,ZNF836,ZNF839,ZNF84,ZNF862,ZNF92,ZNF93,ZNFX1,ZNHIT1,ZNHIT3,ZNHIT6,ZNRD1,ZNRF1,ZNRF2,ZP3,ZRANB1,ZRANB2,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN29,ZSCAN5B,ZSWIM6,ZSWIM7,ZUFSP,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
1,3,6,5,12,RNA,1,whole blood,Homo sapiens,9606,46,4,10,total RNA,Total RNA was isolated in accordance with the ...,biotin,100,"Standard Affymetrix GeneAtlas® Hybridization, ...",Standard GeneTitan® multi channel instrument.,blood gene expression data from healthy subjec...,2.15,13667,"Brendon,,Scicluna",brendon.scicluna@um.edu.mt,University of Malta,Msida campus,Msida,2020,Malta,1602,65682,24646,female,46,1,2.261576,3.104086,3.275543,5.371443,6.391021,4.131982,...,4.127473,3.539460,4.646045,4.363789,2.415078,3.414326,4.759554,3.820201,2.626380,3.530326,3.022236,4.803728,2.607287,2.637789,6.694000,6.964510,3.950472,4.109149,6.604810,4.093587,3.488449,3.140045,4.493763,6.838405,2.968794,3.409196,2.376050,3.544601,2.396704,7.666054,4.786075,3.155967,3.326524,3.874734,3.921279,3.292959,3.826366,6.528229,5.496339,3.752368
2,3,6,5,12,RNA,1,whole blood,Homo sapiens,9606,69,4,10,total RNA,Total RNA was isolated in accordance with the ...,biotin,100,"Standard Affymetrix GeneAtlas® Hybridization, ...",Standard GeneTitan® multi channel instrument.,blood gene expression data from healthy subjec...,2.15,13667,"Brendon,,Scicluna",brendon.scicluna@um.edu.mt,University of Malta,Msida campus,Msida,2020,Malta,1602,65682,24646,male,69,1,2.277492,3.565000,2.753998,5.133350,7.212130,4.197382,...,4.555637,3.513376,4.574943,4.635307,2.883287,3.188046,4.861153,3.919249,3.025347,3.746914,3.177470,4.704257,2.392589,2.632080,5.661406,7.334993,4.239784,3.185535,7.215705,3.511894,3.291209,3.327510,4.638602,7.119833,2.954438,3.349922,2.835412,2.872529,2.262432,7.290612,4.008079,3.854365,3.431031,3.738531,2.793763,5.173026,4.850186,6.490220,5.241529,3.367788
4,3,6,5,12,RNA,1,whole blood,Homo sapiens,9606,64,4,10,total RNA,Total RNA was isolated in accordance with the ...,biotin,100,"Standard Affymetrix GeneAtlas® Hybridization, ...",Standard GeneTitan® multi channel instrument.,blood gene expression data from healthy subjec...,2.15,13667,"Brendon,,Scicluna",brendon.scicluna@um.edu.mt,University of Malta,Msida campus,Msida,2020,Malta,1602,65682,24646,female,64,1,2.161031,3.847223,2.579574,4.917349,5.910861,3.829845,...,4.211501,3.184728,4.089738,3.640608,2.629668,2.905217,4.097917,3.354093,2.666624,3.501176,2.746188,4.627008,2.744753,3.145275,6.047121,6.649186,3.946829,3.756474,6.110988,3.185836,3.711202,3.299832,5.008900,6.708813,3.581132,2.865678,2.162438,3.203983,2.320672,7.906131,3.413840,3.452188,3.152369,3.121356,3.579030,6.004255,5.252546,6.746959,5.662583,2.993148
6,3,6,5,12,RNA,1,whole blood,Homo sapiens,9606,31,4,10,total RNA,Total RNA was isolated in accordance with the ...,biotin,100,"Standard Affymetrix GeneAtlas® Hybridization, ...",Standard GeneTitan® multi channel instrument.,blood gene expression data from healthy subjec...,2.15,13667,"Brendon,,Scicluna",brendon.scicluna@um.edu.mt,University of Malta,Msida campus,Msida,2020,Malta,1602,65682,24646,male,31,0,2.649911,2.873127,2.992352,4.755012,6.412923,4.336686,...,3.834656,4.358438,3.416757,4.064343,3.593553,3.401760,3.216968,3.493291,2.823335,3.858556,3.493064,4.352361,2.531781,2.617447,6.561782,6.136928,2.587619,3.788440,6.652561,3.808909,2.347702,3.001857,4.882089,5.254128,3.201393,5.385193,3.022916,2.985160,2.406831,6.767188,3.114

In [105]:
missing_summary = data.isna().sum()
missing_summary = pd.DataFrame({'count': missing_summary})
missing_summary = missing_summary[missing_summary['count'] > 0]
missing_summary['percent'] = 100 * (missing_summary['count'] / data.shape[0])
missing_summary.sort_values(by='count', ascending=False)

,count,percent


In [ ]:
X = data.drop(['Mortality'], axis=1)
y = data['Mortality'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 2) Preprocess: numeric -> impute+scale, categorical -> impute+onehot(ignore unknown)
num_proc = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler())
])

cat_proc = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_proc, selector(dtype_include=["number"])),
        ("cat", cat_proc, selector(dtype_exclude=["number"]))
    ],
    remainder="drop"   # or "passthrough" if you want untouched columns kept
)

# 3) Choose a model (swap lr/rf/xgb as you like)
lr = LogisticRegression(max_iter=2000, n_jobs=None if hasattr(LogisticRegression(), "n_jobs") else None)
rf = RandomForestClassifier(n_estimators=400, random_state=42)
xgb = XGBClassifier(
    n_estimators=400, max_depth=4, learning_rate=0.05,
    subsample=0.9, colsample_bytree=0.9, random_state=42,
    eval_metric="logloss", tree_method="hist"
)

model = lr  # <- pick: lr / rf / xgb

clf = Pipeline(steps=[("prep", preprocess), ("model", model)])

# 4) Fit & evaluate (use proba for AUC)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1] if hasattr(clf.named_steps["model"], "predict_proba") else None

acc = accuracy_score(y_test, y_pred)
sen = recall_score(y_test, y_pred)  # sensitivity = recall for positive class 1
auc = roc_auc_score(y_test, y_prob) if y_prob is not None else float("nan")

print(f"Accuracy = {acc:.4f}, AUC = {auc:.4f}, Sensitivity = {sen:.4f}")

Accuracy = 1.0000, AUC = 1.0000, Sensitivity = 1.0000


In [ ]:
set(X.columns).intersection([y.name])

set()

In [123]:
df_corr = data.corr(numeric_only=True)[['Mortality']].sort_values('Mortality', ascending=False)
print(df_corr.head(20))

           Mortality
Mortality   1.000000
S100A8      0.846661
S100A12     0.818598
ANXA3       0.762283
ARG1        0.742955
C19orf59    0.727158
TXN         0.713199
C5orf32     0.710656
PGS1        0.685800
IRAK3       0.676826
UPP1        0.675850
SLC22A4     0.664600
TLR5        0.663504
GADD45A     0.659049
CD177       0.656718
LILRA5      0.654466
CD59        0.653648
IL1R2       0.652329
CKAP4       0.652078
CLEC4D      0.650021


In [125]:
data.columns

Index(['title', 'status', 'submission_date', 'last_update_date', 'type',
       'channel_count', 'source_name_ch1', 'organism_ch1', 'taxid_ch1',
       'characteristics_ch1',
       ...
       'ZSWIM7', 'ZUFSP', 'ZW10', 'ZWILCH', 'ZWINT', 'ZXDC', 'ZYG11B', 'ZYX',
       'ZZEF1', 'ZZZ3'],
      dtype='object', length=10883)